# Data Cleaning and Preparation

* During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging.
* Such tasks are often reported to take up 80% or more of an analyst’s time.
* Sometimes the way that data is stored in files or databases is not in the right format for a particular task.
* Many researchers choose to do ad hoc processing of data from one form to another using a general-purpose programming language, like Python, Perl, R, or Java,
or Unix text-processing tools like sed or awk.
* Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.

In [2]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

* Missing data occurs commonly in many data analysis applications.
* One of the goals of pandas is to make working with missing data as painless as possible.
* Example: all of the descriptive statistics on pandas objects exclude missing data by default.
* The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users.
* For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

* In pandas, we’ve adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available.
* In statistics applications, NA data may either be data that does not exist or that exists but was not observed.
* When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data
collection problems or potential biases in the data caused by missing data.
---
* The built-in Python None value is also treated as NA in object arrays:

In [ ]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

* Table below describes a list of some functions related to missing data handling.

<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/19_orig.png">
</p>

### Filtering Out Missing Data

* There are a few ways to filter out missing data. While you always have the option to do it by hand using ***pandas.isnull*** and ***boolean indexing, the dropna*** can be helpful.
* On a Series, it returns the Series with only the non-null data and index values:

In [2]:
from numpy import nan as NA
import pandas as pd
data = pd.Series([1, NA, 3.5, NA, 7])
#data.dropna()

* This is equivalent to:

In [3]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

* With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. dropna by default drops
any row containing a missing value:

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [ ]:
cleaned

,0,1,2
0,1.0,6.5,3.0


* Passing how='all' will only drop rows that are all NA:

In [ ]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


* To drop columns in the same way, pass axis=1:

In [ ]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [ ]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


* A related way to filter out DataFrame rows tends to concern time series data.
* Suppose you want to keep only rows containing a certain number of observations.
* You can indicate this with the thresh argument:

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
df.dropna(thresh=2) # Drops first two rows

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling In Missing Data

* Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways.
* For most purposes, the ***fillna method is the workhorse*** function to use.
* Calling fillna with a constant replaces missing values with that value:



In [ ]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


* Calling fillna with a dict, you can use a different fill value for each column:

In [ ]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


* fillna returns a new object, but you can modify the existing object in-place:

In [ ]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


* The same interpolation methods available for reindexing can be used with fillna:

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [ ]:
df.fillna(method='ffill')

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [ ]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


* With fillna you can do lots of other things with a little creativity.
* Example: you might pass the mean or median value of a Series:

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

*  Table below for a reference on fillna.

<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/20_orig.png">
</p>

## Data Transformation

* So far we’ve been concerned with rearranging data.
* Filtering, cleaning, and other transformations are another class of important operations.

### Removing Duplicates

* Duplicate rows may be found in a DataFrame for any number of reasons.
* Here is an example:

In [7]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


* The DataFrame method duplicated returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [8]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

* Relatedly, drop_duplicates returns a DataFrame where the duplicated array is False:

In [9]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


* Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates.
* Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [10]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [11]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


* **duplicated** and **drop_duplicates** by default keep the first observed value combination.
* Passing keep='**last'** will return the last one:

In [12]:
data.drop_duplicates(['k1', 'k2'], keep='last') # 5th index will be dropped

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

* For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame.
* Consider the following hypothetical data collected about various kinds of dish:

In [ ]:
data = pd.DataFrame({'food': ['penne pasta', 'pizza', 'penne pasta',
                              'neer dosa', 'ghee dosa', 'Penne Pasta',
                              'neer dosa', 'alfredo pasta', 'Butter Dosa'],
                     'quantity': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,quantity
0,penne pasta,4.0
1,pizza,3.0
2,penne pasta,12.0
3,neer dosa,6.0
4,ghee dosa,7.5
5,Penne Pasta,8.0
6,neer dosa,3.0
7,alfredo pasta,5.0
8,Butter Dosa,6.0


* Suppose you wanted to add a column indicating the place that each food
came from.
* Let’s write down a mapping of each distinct food type to the place:

In [ ]:
food_to_place = {
  'penne pasta': 'italin',
  'pizza': 'italin',
  'neer dosa': 'udupi',
  'ghee dosa': 'udupi',
  'alfredo pasta': 'italin',
  'butter dosa': 'davangere'
}

* The map method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the food are capitalized and
others are not.
* Thus, we need to convert each value to lowercase using the ***str.lower*** Series method:

In [ ]:
lowercased = data['food'].str.lower()
lowercased

0      penne pasta
1            pizza
2      penne pasta
3        neer dosa
4        ghee dosa
5      penne pasta
6        neer dosa
7    alfredo pasta
8      butter dosa
Name: food, dtype: object

In [ ]:
data['place'] = lowercased.map(food_to_place)
data

,food,quantity,place
0,penne pasta,4.0,italin
1,pizza,3.0,italin
2,penne pasta,12.0,italin
3,neer dosa,6.0,udupi
4,ghee dosa,7.5,udupi
5,Penne Pasta,8.0,italin
6,neer dosa,3.0,udupi
7,alfredo pasta,5.0,italin
8,Butter Dosa,6.0,davangere


* We could also have passed a function that does all the work:

In [ ]:
data['food'].map(lambda x: food_to_place[x.lower()])

0       italin
1       italin
2       italin
3        udupi
4        udupi
5       italin
6        udupi
7       italin
8    davangere
Name: food, dtype: object

* Using ***map*** is a convenient way to perform element-wise transformations and other
data cleaning related operations.

### Replacing Values

* Filling in missing data with the ***fillna*** method is a special case of more general value replacement.
As you’ve already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so.
* Let’s consider this Series:

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

* The -999 values might be sentinel values for missing data.
* To replace these with NA values that pandas understands, we can use ***replace, producing a new Series*** (unless
you pass inplace=True):

In [ ]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

* If you want to replace multiple values at once, you instead pass a list and then the
substitute value:

In [ ]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

* To use a different replacement for each value, pass a list of substitutes:

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

* The argument passed can also be a dict:

In [ ]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

* Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.
* You can also modify the axes in-place without creating a new data structure. Here’s a simple example:

In [14]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


* Like a Series, the axis indexes have a map method:

In [15]:
transform = lambda x: x[:4].upper() # Here :4 means first 4 characters
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

* You can assign to index, modifying the DataFrame in-place:

In [16]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


* If you want to create a transformed version of a dataset without modifying the original, a useful method is ***rename***:

In [ ]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


* Notably, ***rename*** can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


*  If you wish to modify a dataset in-place, pass inplace=True:

In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

* Continuous data is often discretized or otherwise separated into “bins” for analysis.
* Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets:

In [18]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

* Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older.
* To do so, you have to use ***cut***, a function in pandas:

In [19]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

* The object pandas returns is a special ***Categorical object***.
* The output you see describes the bins computed by ***pandas.cut***.
* You can treat it like an array of strings
indicating the bin name; internally it contains a categories array specifying the distinct category names along with a labeling for the ages data in the codes attribute:

In [23]:
cats.codes # specify to which age group the given age belongs

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [21]:
cats.categories #specify the categories created

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [ ]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

* Note that ***pd.value_counts(cats)*** are the bin counts for the result of ***pandas.cut***.

---
* Consistent with mathematical notation for intervals, a parenthesis means that the side
is open, while the square bracket means it is closed (inclusive).
* You can change which side is closed by passing ***right=False***:

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

* You can also pass your own bin names by passing a list or array to the ***labels*** option:

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

* If you pass an integer number of bins to cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.
* Consider the case of some uniformly distributed data chopped into fourths:

In [25]:
data = np.random.rand(20)
data

array([0.6425, 0.7175, 0.4676, 0.3256, 0.4396, 0.7297, 0.994 , 0.6769,
       0.7908, 0.1709, 0.0268, 0.8004, 0.9037, 0.0247, 0.4917, 0.5263,
       0.5964, 0.052 , 0.8951, 0.7283])

In [26]:
pd.cut(data, 4, precision=2)

[(0.51, 0.75], (0.51, 0.75], (0.27, 0.51], (0.27, 0.51], (0.27, 0.51], ..., (0.51, 0.75], (0.51, 0.75], (0.024, 0.27], (0.75, 0.99], (0.51, 0.75]]
Length: 20
Categories (4, interval[float64, right]): [(0.024, 0.27] < (0.27, 0.51] < (0.51, 0.75] <
                                           (0.75, 0.99]]

* The precision=2 option limits the decimal precision to two digits.
---
* A closely related function, ***qcut***, bins the data based on sample quantiles.
* Depending on the distribution of the data, using cut will not usually result in each bin having the same number of data points.
* Since ***qcut*** uses sample quantiles instead, by definition you will obtain roughly equal-size bins:

In [ ]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(0.626, 3.928], (0.626, 3.928], (-2.9499999999999997, -0.691], (-0.691, -0.0171], (0.626, 3.928], ..., (-0.0171, 0.626], (0.626, 3.928], (-0.691, -0.0171], (-0.691, -0.0171], (-0.0171, 0.626]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.691] < (-0.691, -0.0171] < (-0.0171, 0.626] <
                                           (0.626, 3.928]]

In [ ]:
pd.value_counts(cats)

(-2.9499999999999997, -0.691]    250
(-0.691, -0.0171]                250
(-0.0171, 0.626]                 250
(0.626, 3.928]                   250
dtype: int64

* Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(1.297, 3.928], (-0.0171, 1.297], (-2.9499999999999997, -1.191], (-1.191, -0.0171], (1.297, 3.928], ..., (-0.0171, 1.297], (-0.0171, 1.297], (-1.191, -0.0171], (-1.191, -0.0171], (-0.0171, 1.297]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -1.191] < (-1.191, -0.0171] < (-0.0171, 1.297] <
                                           (1.297, 3.928]]

### Detecting and Filtering Outliers

* Filtering or transforming outliers is largely a matter of applying array operations.
* Consider a DataFrame with some normally distributed data:

In [35]:
data = pd.DataFrame(np.random.randn(1000, 4))
data

,0,1,2,3
0,-0.738076,0.387475,-0.297376,-1.322115
1,1.258306,-1.130313,0.385064,-0.693637
2,0.966616,-0.374624,1.229986,-0.900868
3,0.571032,-2.578745,0.952680,-0.065575
4,0.735289,-1.311494,-0.801136,-1.085607
...,...,...,...,...
995,0.670572,1.692452,1.440670,0.633386
996,0.925563,0.075544,-0.689847,-0.160437
997,0.730806,-0.660930,-0.338339,0.750402
998,1.374380,2.436133,1.331873,1.834021


In [36]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.036164,-0.028239,-0.011713,-0.026345
std,1.004734,1.000669,0.989712,1.015033
min,-3.530912,-3.024110,-3.170292,-3.105636
25%,-0.737059,-0.692298,-0.692553,-0.752602
50%,0.011917,-0.011204,0.021330,-0.028327
75%,0.667138,0.609290,0.668412,0.705510
max,2.916153,3.061029,2.623689,3.144389


* Suppose you wanted to find values in one of the columns exceeding 3 in absolute
value:

In [29]:
col = data[2]
col[np.abs(col) > 3]

95     3.176873
317    3.260383
492   -3.056990
579   -3.184377
Name: 2, dtype: float64

* To select all rows having a value exceeding 3 or –3, you can use the ***any method on a boolean DataFrame***:

In [30]:
data[(np.abs(data) > 3).any(1)]

<ipython-input-30-972b47795d6b>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data[(np.abs(data) > 3).any(1)]


,0,1,2,3
90,-0.774363,0.552936,0.106061,3.927528
95,-0.655054,-0.565230,3.176873,0.959533
298,-2.315555,0.457246,-0.025907,-3.399312
317,0.050188,1.951312,3.260383,0.963301
393,0.146326,0.508391,-0.196713,-3.745356
492,-0.293333,-0.242459,-3.056990,1.918403
516,-3.428254,-0.296336,-0.439938,-0.867165
579,0.275144,1.179227,-3.184377,1.369891
801,-0.362528,-3.548824,1.553205,-2.186301
893,3.366626,-2.372214,0.851010,1.332846


* Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:

In [32]:
data[np.abs(data) > 3] = np.sign(data) * 3
data

,0,1,2,3
0,0.000940,1.343810,-0.713544,-0.831154
1,-2.370232,-1.860761,-0.860757,0.560145
2,-1.265934,0.119827,-1.063512,0.332883
3,-2.359419,-0.199543,-1.541996,-0.970736
4,-1.307030,0.286350,0.377984,-0.753887
...,...,...,...,...
995,-0.153514,0.011194,-0.050555,0.420211
996,1.190981,1.561488,-1.132925,0.097083
997,-1.031573,-0.044618,-0.218153,0.772942
998,-0.728643,0.215523,-2.056737,-1.248733


In [33]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.066009,0.067564,0.013430,-0.000242
std,0.994043,0.990965,1.000047,0.989367
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.596286,-0.670407,-0.642609
50%,-0.116401,0.094503,-0.005700,-0.012007
75%,0.619779,0.780282,0.671128,0.655516
max,3.000000,2.653656,3.000000,3.000000


* The statement **np.sign(data) produces 1 and –1 values based on whether the values** in data are positive or negative:

In [37]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,1.0,-1.0,1.0,-1.0
2,1.0,-1.0,1.0,-1.0
3,1.0,-1.0,1.0,-1.0
4,1.0,-1.0,-1.0,-1.0


### Permutation and Random Sampling

* Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do
using the ***numpy.random.permutation*** function.
* Calling ***permutation*** with the length
of the axis you want to permute produces an array of integers indicating the new
ordering:

In [38]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [39]:
sampler = np.random.permutation(5)
sampler

array([2, 0, 3, 4, 1])

* That array can then be used in iloc-based indexing or the equivalent take function:

In [40]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
0,0,1,2,3
3,12,13,14,15
4,16,17,18,19
1,4,5,6,7


* To select a ***random subset without replacement***, you can use the sample method on Series and DataFrame:

In [42]:
df.sample(n=3) # Every time you execute this block you will get different rows (Thus, random)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3


* To generate a sample with replacement (to allow repeat choices), pass replace=True
to sample:

In [43]:
choices = pd.Series([5, 7, -1, 6, 4])
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [44]:
draws = choices.sample(n=10, replace=True)
draws

3    6
0    5
4    4
4    4
1    7
3    6
4    4
4    4
3    6
3    6
dtype: int64

## String Manipulation

* Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing.
* Most text operations are made simple with the string object’s built-in methods.
* For more complex pattern matching and text manipulations, regular expressions may be needed.
* pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

### String Object Methods

* In many string manipulation and scripting applications, built-in string methods are sufficient.
* Ex: a comma-separated string can be broken into pieces with ***split***:

In [8]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

* ***split*** is often combined with ***strip*** to trim whitespace (including line breaks):

In [9]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

* These substrings could be concatenated together with a two-colon delimiter using addition:

In [10]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

* But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the join method on the string ***'::'***:

In [11]:
'::'.join(pieces)

'a::b::guido'

* Other methods are concerned with locating substrings.
* Using Python’s in keyword is
the best way to detect a substring, though index and find can also be used:

In [12]:
'guido' in val

True

In [13]:
val.index(' ')

4

In [15]:
val.find(',')

1

* Note the difference between find and index is that index raises an exception if the string isn’t found (versus returning –1):

In [16]:
val.index(':')

ValueError: substring not found

* Relatedly, ***count*** returns the number of occurrences of a particular substring:

In [18]:
val.count('a')

1

* ***replace*** will substitute occurrences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:

In [ ]:
val.replace(',', '::')

'a::b::  guido'

In [19]:
val.replace(',', '')

'ab  guido'

In [21]:
val.endswith('a')

False

In [22]:
val.startswith('a')

True

In [25]:
val.find(',')

1

In [30]:
val_new=val.upper()

In [40]:
val_n='2nidhi'
val_n.ljust(10,' ')

'2nidhi    '

* Table below give a listing of some of Python’s string methods.

<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/21_orig.png">
</p>

### Regular Expressions

* Regular expressions provide a flexible way to search or match (often more complex) string patterns in text.
* A single expression, commonly called a regex, is a string formed according to the regular expression language.
* Python’s built-in ***re*** module is
responsible for applying regular expressions to strings.
---
* The **re** module functions fall into three categories: ***pattern matching, substitution, and splitting***.
* Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes.
* Use Case: suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines). The regex describing one or more whitespace characters
is ***\s+***:


In [43]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

* When you call *re.split('\s+', text)*, the regular expression is first compiled, and then its split method is called on the passed text.
* You can compile the regex yourself
with re.compile, forming a reusable regex object:

In [48]:
regex = re.compile('\s+')
regex
regex.split(text)


['foo', 'bar', 'baz', 'qux']

* If, instead, you wanted to get a list of all patterns matching the regex, you can use the findall method:

In [57]:
re.findall('\s+',text)

['    ', '\t ', '  \t']

* Creating a regex object with re.compile is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles.
---
* ***match*** and ***search*** are closely related to ***findall***.
* While findall returns all matches in a string, search returns only the first match. More rigidly, match only matches at the beginning of the string.
* USe Case: let’s consider a block of text and a regular expression capable of identifying most email addresses:

In [65]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [59]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9]+.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)


* Using findall on the text produces a list of the email addresses:

In [60]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

* search returns a special match object for the first email address in the text.
* For the preceding regex, the match object can only tell us the start and end position of the pattern in the string:

In [61]:
m=re.search('@',text)
m

<re.Match object; span=(9, 10), match='@'>

In [66]:
text[m.start():]

'@google.com\nSteve steve@gmail.com\nRob rob@gmail.com\nRyan ryan@yahoo.com\n'

* regex.match returns None, as it only will match if the pattern occurs at the start of the string:

In [70]:
print(re.match('D',text))

<re.Match object; span=(0, 1), match='D'>


* Relatedly, sub will return a new string with occurrences of the pattern replaced by the a new string:

In [70]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



* Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix.
* To do this, put parentheses around the parts of the pattern to segment:

In [72]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

* A match object produced by this modified regex returns a tuple of the pattern components with its groups method:

In [72]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [73]:
text

'Dave dave@google.com\nSteve steve@gmail.com\nRob rob@gmail.com\nRyan ryan@yahoo.com\n'

* findall returns a list of tuples when the pattern has groups:

In [74]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

* sub also has access to groups in each match using special symbols like \1 and \2.
* The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:

In [75]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



* Table below provides a brief summary for regular expressions in Python.

<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/24_orig.png">
</p>

### Vectorized String Functions in pandas

* Cleaning up a messy dataset for analysis often requires a lot of string manipulation and regularization.
* To complicate matters, a column containing strings will sometimes have missing data:

In [75]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [76]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

* You can apply string and regular expression methods (passing a ***lambda*** or other function) to each value using data.map, but it will fail on the NA (null) values.
* To cope with this, Series has array-oriented methods for string operations that skip NA values.
* These are accessed through Series’s str attribute;
* Ex: we could check whether each email address has 'gmail' in it with ***str.contains***:

In [77]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

* Regular expressions can be used, too, along with any re options like ***IGNORECASE***:

In [78]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [79]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

* There are a couple of ways to do vectorized element retrieval. Either use str.get or index into the str attribute:

In [80]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

* You can similarly slice strings using this syntax:

In [ ]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

*  Table below shows the pandas string methods

<p align="center">
    <img src="http://raghudathesh.weebly.com/uploads/4/8/9/6/48968251/25_orig.png">
</p>